<a href="https://colab.research.google.com/github/danflo71/La_Liga_Soccer_Matches/blob/main/La_Liga_WebScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

In [2]:
fixure_standings_url = "https://fbref.com/en/comps/12/La-Liga-Stats"

In [3]:

data = requests.get(fixure_standings_url)

In [4]:

from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(data.text)
fixure_standings_table = soup.select('table.stats_table')[0]
links = fixure_standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [6]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [7]:
data = requests.get(team_urls[0])

In [8]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

<ipython-input-8-391eeaaee4f3>:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


In [9]:

soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [10]:
data = requests.get(f"https://fbref.com{links[0]}")

In [11]:
shooting = pd.read_html(data.text, match="Shooting")[0]

<ipython-input-11-8da1f2d296b3>:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


In [12]:
shooting.head()

For Barcelona                                                           \
           Date   Time     Comp        Round  Day Venue Result   GF   GA   
0    2024-08-17  21:30  La Liga  Matchweek 1  Sat  Away      W  2.0  1.0   
1    2024-08-24  19:00  La Liga  Matchweek 2  Sat  Home      W  2.0  1.0   
2    2024-08-27  21:30  La Liga  Matchweek 3  Tue  Away      W  2.0  1.0   
3    2024-08-31  17:00  La Liga  Matchweek 4  Sat  Home      W  7.0  0.0   
4    2024-09-15  16:15  La Liga  Matchweek 5  Sun  Away      W  4.0  1.0   

                   ... Standard             Expected                    \
         Opponent  ...     Dist FK PK PKatt       xG npxG npxG/Sh G-xG   
0        Valencia  ...     18.6  1  1     1      3.2  2.4    0.14 -1.2   
1   Athletic Club  ...     16.6  0  0     0      1.8  1.8    0.14  0.2   
2  Rayo Vallecano  ...     19.3  1  0     0      1.4  1.4    0.06  0.6   
3      Valladolid  ...     13.7  1  0     0      4.7  4.7    0.21  2.3   
4          Girona  ...     19.1  0  0     0      1.9  1.9    0.09  2.1   

          Unnamed: 25_level_0  
  np:G-xG        Match Report  
0    -1.4        Match Report  
1     0.2        Match Report  
2     0.6        Match Report  
3     2.3        Match Report  
4     2.1        Match Report  

[5 rows x 26 columns]

In [13]:
shooting.columns = shooting.columns.droplevel()

In [14]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [15]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Opp Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2024-08-17,21:30,La Liga,Matchweek 1,Sat,Away,W,2.0,1.0,Valencia,...,4-4-2,NaN,Match Report,NaN,17,5,18.6,1,1,1
1,2024-08-24,19:00,La Liga,Matchweek 2,Sat,Home,W,2.0,1.0,Athletic Club,...,4-2-3-1,NaN,Match Report,NaN,13,5,16.6,0,0,0
2,2024-08-27,21:30,La Liga,Matchweek 3,Tue,Away,W,2.0,1.0,Rayo Vallecano,...,4-4-2,NaN,Match Report,NaN,22,5,19.3,1,0,0
3,2024-08-31,17:00,La Liga,Matchweek 4,Sat,Home,W,7.0,0.0,Valladolid,...,4-1-4-1,Isidro Díaz de Mera,Match Report,NaN,23,11,13.7,1,0,0
4,2024-09-15,16:15,La Liga,Matchweek 5,Sun,Away,W,4.0,1.0,Girona,...,4-2-3-1,Alejandro Muñíz,Match Report,NaN,20,9,19.1,0,0,0


In [16]:

years = list(range(2025, 2023, -1))
all_matches = []

In [17]:

fixure_standings_url = "https://fbref.com/en/comps/12/La-Liga-Stats"

In [18]:
import time
for year in years:
    data = requests.get(fixure_standings_url)
    soup = BeautifulSoup(data.text)
    fixure_standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in fixure_standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "La Liga"]

        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(10)

<ipython-input-18-b09cb08a2b15>:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
<ipython-input-18-b09cb08a2b15>:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
<ipython-input-18-b09cb08a2b15>:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
<ipython-input-18-b09cb08a2b15>:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO

In [19]:
len(all_matches)

40

In [20]:

match_df = pd.concat(all_matches)

In [21]:

match_df.columns = [c.lower() for c in match_df.columns]

In [22]:
match_df


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2024-08-17,21:30,La Liga,Matchweek 1,Sat,Away,W,2.0,1.0,Valencia,...,Match Report,NaN,17,5,18.6,1.0,1,1,2025,Barcelona
1,2024-08-24,19:00,La Liga,Matchweek 2,Sat,Home,W,2.0,1.0,Athletic Club,...,Match Report,NaN,13,5,16.6,0.0,0,0,2025,Barcelona
2,2024-08-27,21:30,La Liga,Matchweek 3,Tue,Away,W,2.0,1.0,Rayo Vallecano,...,Match Report,NaN,22,5,19.3,1.0,0,0,2025,Barcelona
3,2024-08-31,17:00,La Liga,Matchweek 4,Sat,Home,W,7.0,0.0,Valladolid,...,Match Report,NaN,23,11,13.7,1.0,0,0,2025,Barcelona
4,2024-09-15,16:15,La Liga,Matchweek 5,Sun,Away,W,4.0,1.0,Girona,...,Match Report,NaN,20,9,19.1,0.0,0,0,2025,Barcelona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,2024-08-25,19:00,La Liga,Matchweek 2,Sun,Away,L,1.0,2.0,Leganés,...,Match Report,NaN,10,5,16.6,0.0,0,0,2024,Las Palmas
2,2024-08-29,20:30,La Liga,Matchweek 3,Thu,Home,D,1.0,1.0,Real Madrid,...,Match Report,NaN,7,2,18.1,0.0,0,0,2024,Las Palmas
3,2024-09-01,17:00,La Liga,Matchweek 4,Sun,Away,L,0.0,2.0,Alavés,...,Match Report,NaN,12,6,18.4,1.0,0,0,2024,Las Palmas
4,2024-09-15,17:30,La Liga,Matchweek 5,Sun,Home,L,2.0,3.0,Athletic Club,...,Match Report,NaN,15,5,17.9,2.0,0,0,2024,Las Palmas
